In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from mpmath import besseljzero
from scipy.special import gamma
from scipy.special import jv, iv
from scipy.optimize import differential_evolution
from scipy.interpolate import interp1d

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simulate_HSDM_2D(a, mu, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    while np.linalg.norm(x, 2) < a(rt):
        x += mu*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta = np.arctan2(x[1], x[0])   
    
    return ndt+rt, theta

In [3]:
def series_bessel_fpt(t, a=1, sigma=1, nu=0, n=100):
    zeros = np.asarray([float(besseljzero(nu, i+1)) for i in range(n)])
    fpt = np.zeros(t.shape)
    
    for i in range(t.shape[0]):
        series = np.sum((zeros**(nu+1)/jv(nu+1, zeros)) * np.exp(-(zeros**2 * sigma**2)/(2*a**2)*t[i]))
        fpt[i] = sigma**2/(2**nu * a**2 * gamma(nu + 1)) * series
        
    return interp1d(t, fpt)

In [4]:
def HSDM_2D_likelihood(prms, RT, Theta, N_series):
    a =  prms[0]
    ndt = prms[1]
    mu = np.array([prms[2], prms[3]])
    
    tt = np.arange(0.001, max(RT)+0.001, 0.001)
    fpt = series_bessel_fpt(tt, a, sigma=1, nu=(mu.shape[0]-2)/2, n=N_series)
    
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.001:
            mu_dot_x0 = mu[0]*np.cos(theta)
            mu_dot_x1 = mu[1]*np.sin(theta)
            term1 = prms[0] * (mu_dot_x0 + mu_dot_x1)
            term2 = 0.5 * np.linalg.norm(mu, 2)**2 * (rt - ndt)
            
            density = np.exp(term1 - term2) * fpt(rt - ndt)
            
            if 0.1**14 < density:
                log_lik += -np.log(density)
            else:
                log_lik += -np.log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [5]:
recovery_df = {'threshold_true': [],
               'threshold_estimate': [],
               'ndt_true': [],
               'ndt_estimate': [],
               'mu1_true': [],
               'mu1_estimate': [],
               'mu2_true': [],
               'mu2_estimate': []}

min_threshold = 0.5
max_threshold = 6

min_ndt = 0.1
max_ndt = 1

min_mu = -6
max_mu = 6

N_series = 50

In [6]:
for n in tqdm(range(20)):
    threshold = np.random.uniform(min_threshold, max_threshold)
    a = lambda t: threshold
    ndt = np.random.uniform(min_ndt, max_ndt)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu)])
    
    
    recovery_df['threshold_true'].append(threshold)
    recovery_df['ndt_true'].append(ndt)
    recovery_df['mu1_true'].append(mu[0])
    recovery_df['mu2_true'].append(mu[1])
    
    RT = []
    Theta = []
    
    for i in range(250):
        rt, theta = simulate_HSDM_2D(a, mu, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    min_ans = differential_evolution(HSDM_2D_likelihood,
                                     args=(RT, Theta, N_series),
                                     bounds=[(min_threshold, max_threshold), (min_ndt, max_ndt),
                                             (min_mu, max_mu), (min_mu, max_mu)])
    
    recovery_df['threshold_estimate'].append(min_ans.x[0])
    recovery_df['ndt_estimate'].append(min_ans.x[1])
    recovery_df['mu1_estimate'].append(min_ans.x[2])
    recovery_df['mu2_estimate'].append(min_ans.x[3])
    
recovery_df = pd.DataFrame(recovery_df)

100%|██████████████████████████████████████████| 20/20 [37:55<00:00, 113.76s/it]


In [7]:
recovery_df

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate
0,5.284266,5.188260,0.958298,0.964914,-5.326955,-5.200239,3.853295,3.771644
1,2.888399,2.926201,0.367338,0.340875,-0.508830,-0.608709,-2.806777,-2.729825
2,2.200772,2.103741,0.907816,0.940054,-1.861620,-1.937444,-3.968196,-3.950957
3,4.020522,3.718937,0.310865,0.349248,5.856246,5.756528,-4.733474,-4.773165
4,0.768389,0.725386,0.694572,0.710733,-2.545798,-2.440582,-1.905746,-1.852758
5,3.437042,3.574751,0.723774,0.685344,0.666793,0.492802,4.629428,4.529011
6,2.809457,3.117842,0.543382,0.504732,2.578860,2.710359,-3.921149,-3.999564
7,4.002184,3.542532,0.371308,0.597814,-1.345956,-1.371625,1.382913,1.370163
8,5.545921,5.571887,0.504930,0.450616,-5.435585,-5.041753,4.962220,4.628316
9,1.791284,1.802960,0.967437,0.965905,2.444919,2.519576,4.365774,4.250658


In [8]:
recovery_df.corr()

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate
threshold_true,1.000000,0.994223,-0.114581,-0.085075,-0.083370,-0.080131,0.216635,0.202569
threshold_estimate,0.994223,1.000000,-0.086804,-0.072244,-0.069192,-0.065588,0.210798,0.196236
ndt_true,-0.114581,-0.086804,1.000000,0.982946,-0.126659,-0.145486,-0.111196,-0.119096
ndt_estimate,-0.085075,-0.072244,0.982946,1.000000,-0.136655,-0.157426,-0.112070,-0.119350
mu1_true,-0.083370,-0.069192,-0.126659,-0.136655,1.000000,0.998876,-0.349735,-0.345233
mu1_estimate,-0.080131,-0.065588,-0.145486,-0.157426,0.998876,1.000000,-0.332009,-0.327114
mu2_true,0.216635,0.210798,-0.111196,-0.112070,-0.349735,-0.332009,1.000000,0.999550
mu2_estimate,0.202569,0.196236,-0.119096,-0.119350,-0.345233,-0.327114,0.999550,1.000000


In [9]:
file_name = 'Series_2d_recovery_{}.csv'.format(N_series)
# old_recovery_data = pd.read_csv(file_name, index_col=0)
# recovery_df = pd.concat([old_recovery_data, 
#                          recovery_df]).reset_index(drop=True)
recovery_df.to_csv(file_name)

In [18]:
%time series_bessel_fpt(np.arange(0.001, 5, 0.1), a=1, sigma=1, nu=0, n=500)


CPU times: user 997 ms, sys: 1.29 ms, total: 998 ms
Wall time: 998 ms
